# rhino3dm exercise 02

## read `.3dm` file

In [1]:
import rhino3dm

In [2]:
rhino_f_path = "./data/tw_nongke.3dm"
file3dm_obj = rhino3dm.File3dm.Read(rhino_f_path)
print('rhino3dm.File3dm object: ', file3dm_obj)

rhino3dm.File3dm object:  <rhino3dm._rhino3dm.File3dm object at 0x00000120859DD5B0>

## read `notes`

In [3]:
rhino3dm_notes = rhino3dm.File3dm.ReadNotes(rhino_f_path).split(';')[:-1]

for idx, note in enumerate(rhino3dm_notes):
    print(
        note.strip().split(':')[0].strip(),
        ' : ',
        note.strip().split(':')[1].strip()
    )

name  :  TW_nongke
lat  :  22.71610997672558
lon  :  120.53574162642127

> **File3dm.ReadNotes()**
>
> [***method***
> `rhino3dm.File3dm.ReadNotes()`](https://mcneel.github.io/rhino3dm/python/api/File3dm.html#rhino3dm.File3dm.ReadNotes)
>
> <br />
>
> Reads only the notes from an existing 3dm file.
>
> <br />
>
> Parameters:
>
> -   path (str) – The file from which to read the notes.
>
> Returns:
>
> -   The 3dm file notes.
>
> Return type:
>
> -   str

## read `Layer`

In [4]:
layer_table = file3dm_obj.Layers
print('layer table object: ', layer_table)
print('num of layers: ', len(layer_table))
print()

for idx, layer in enumerate(iter(layer_table)):
    print('layer ', idx, ' Object  : ', layer)
    print('      ', idx, ' Id      : ', layer.Id)
    print('      ', idx, ' Index   : ', layer.Index)
    print('      ', idx, ' Name    : ', layer.Name)
    print('      ', idx, ' Fullpath: ', layer.FullPath)
    print('      ', idx, ' Color   : ', layer.Color)
    print()

layer table object:  <rhino3dm._rhino3dm.File3dmLayerTable object at 0x00000120AF3B0C30>
num of layers:  4

layer  0  Object  :  <rhino3dm._rhino3dm.Layer object at 0x00000120AF844F70>
       0  Id      :  212960d6-ad46-439d-8516-772ce85fde03
       0  Index   :  0
       0  Name    :  0
       0  Fullpath:  0
       0  Color   :  (0, 0, 0, 255)

layer  1  Object  :  <rhino3dm._rhino3dm.Layer object at 0x00000120AF3790F0>
       1  Id      :  25488de8-55a5-470b-a098-ea3c8c1220b3
       1  Index   :  1
       1  Name    :  cable
       1  Fullpath:  cable
       1  Color   :  (255, 127, 0, 255)

layer  2  Object  :  <rhino3dm._rhino3dm.Layer object at 0x0000012085A03BF0>
       2  Id      :  0428f404-44e4-47e7-ae60-72284d0a89f7
       2  Index   :  2
       2  Name    :  pv
       2  Fullpath:  pv
       2  Color   :  (0, 0, 0, 255)

layer  3  Object  :  <rhino3dm._rhino3dm.Layer object at 0x0000012085A7F970>
       3  Id      :  6f40c8fc-3378-4efb-a590-0af0bf5a85dc
       3  Index   : 

## read `Objects` on the `pv` layer

In [5]:
object_table = file3dm_obj.Objects
print('object table: ', object_table, '\n')

for idx, obj in enumerate(iter(object_table)):
    if idx < 77 and obj.Attributes.LayerIndex == 2:
        print('object ', idx, ' Object                : ', obj)
        print('       ', idx, ' Geometry              : ', obj.Geometry)
        print('       ', idx, ' Attributes            : ', obj.Attributes)
        print('       ', idx, ' Attributes.Id         : ', obj.Attributes.Id)
        print('       ', idx, ' Attributes.Name       : ', obj.Attributes.Name)
        print('       ', idx, ' Attributes.LayerIndex : ', obj.Attributes.LayerIndex)
        print('       ', idx, ' Attributes.ObjectColor: ', obj.Attributes.ObjectColor)
        print('       ', idx, ' Attributes.Visible    : ', obj.Attributes.Visible)
        print()
    else:
        pass

object table:  <rhino3dm._rhino3dm.File3dmObjectTable object at 0x00000120AF36BFB0> 

object  74  Object                :  <rhino3dm._rhino3dm.File3dmObject object at 0x0000012085A7ABF0>
        74  Geometry              :  <rhino3dm._rhino3dm.PolylineCurve object at 0x00000120AF36AAB0>
        74  Attributes            :  <rhino3dm._rhino3dm.ObjectAttributes object at 0x00000120AF36AAB0>
        74  Attributes.Id         :  36f08c55-5986-4aa2-b0b8-ae56b26b043c
        74  Attributes.Name       :  
        74  Attributes.LayerIndex :  2
        74  Attributes.ObjectColor:  (0, 0, 0, 255)
        74  Attributes.Visible    :  True

object  75  Object                :  <rhino3dm._rhino3dm.File3dmObject object at 0x00000120AD5C1AB0>
        75  Geometry              :  <rhino3dm._rhino3dm.PolylineCurve object at 0x0000012085A7ABF0>
        75  Attributes            :  <rhino3dm._rhino3dm.ObjectAttributes object at 0x0000012085A7ABF0>
        75  Attributes.Id         :  2a5e0e48-e0ca-4d27-